In [1]:
import tonic
import numpy as np
import timeit
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [2]:
def create_random_input(
    sensor_size=(200, 100, 2),
    n_events=100000,
    dtype=np.dtype([("x", int), ("y", int), ("t", int), ("p", int)]),
):
    """
    Creates random events for testing purposes.
    Returns
    - events - 10k events
    - sensor_size - 200 x 100 x 2 (w,h,pol)
    """

    assert "x" and "t" and "p" in dtype.names

    events = np.zeros(n_events, dtype=dtype)
    events["x"] = np.random.rand(n_events) * sensor_size[0]
    events["p"] = np.random.rand(n_events) * sensor_size[2]
    # sort timestamps to ensure the times are sequential
    events["t"] = np.sort(np.random.rand(n_events) * 1e6)

    if "y" in dtype.names:
        events["y"] = np.random.rand(n_events) * sensor_size[1]

    return events, sensor_size

In [3]:
events, shape = create_random_input()

In [4]:
events

array([( 36, 83,     21, 1), ( 96, 95,     52, 0), ( 18, 71,     70, 0),
       ..., (  4, 11, 999961, 1), (115, 61, 999979, 0),
       (140, 60, 999983, 1)],
      dtype=[('x', '<i8'), ('y', '<i8'), ('t', '<i8'), ('p', '<i8')])

In [5]:
events.shape

(100000,)

In [6]:
frame_transform = tonic.transforms.ToFrame(sensor_size=shape, n_time_bins=1000)

def drop_pixel_in_frame(frames, pixel_coordinates, inplace=False):
    if inplace:
        frame_copy = frames
    else:
        frame_copy = frames.copy()
    for (x,y) in pixel_coordinates:
        frame_copy[:,:,x,y] = 0
    return frame_copy

## Measure the time it takes to drop pixel coordinates from a numpy tensor, where events are encoded as frames/raster

In [7]:
n_repetitions = 20
n_pixels_dropped = np.arange(0,100)

In [8]:
frame_timings = []
for n_pixels in tqdm(n_pixels_dropped):
    pixel_coordinates = [(53,37),]*n_pixels
    frames = frame_transform(events)
    result = timeit.Timer(lambda: drop_pixel_in_frame(frames, pixel_coordinates)).timeit((n_repetitions)) / n_repetitions
    frame_timings.append(result*1000)

  0%|          | 0/100 [00:00<?, ?it/s]

In [9]:
frame_timings_inplace = []
for n_pixels in tqdm(n_pixels_dropped):
    pixel_coordinates = [(53,37),]*n_pixels
    frames = frame_transform(events)
    result = timeit.Timer(lambda: drop_pixel_in_frame(frames, pixel_coordinates, inplace=True)).timeit((n_repetitions)) / n_repetitions
    frame_timings_inplace.append(result*1000)

  0%|          | 0/100 [00:00<?, ?it/s]

## Measure the time it takes to drop pixel coordinates from a numpy array of events

In [10]:
event_timings = []
for n_pixels in tqdm(n_pixels_dropped):
    pixel_coordinates = [(53,37),]*n_pixels
    drop_pixels = tonic.transforms.DropPixel(pixel_coordinates)

    result = timeit.Timer(lambda: drop_pixels(events)).timeit((n_repetitions)) / n_repetitions
    event_timings.append(result*1000)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
plt.plot(n_pixels_dropped, event_timings, label="Pixels dropped directly from event numpy array")
plt.plot(n_pixels_dropped, frame_timings, label="Pixels dropped from copy of frames on CPU")
plt.plot(n_pixels_dropped, frame_timings_inplace, label="Pixels dropped from frames on CPU in-place")
plt.ylabel('miliseconds')
plt.xlabel('number of events dropped')
plt.legend()
plt.savefig("drop_pixel_benchmark.png")